## Run Img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import glob

tf.compat.v1.reset_default_graph()
interpreter = tf.lite.Interpreter('/content/drive/MyDrive/Colab Notebooks/efficientdet/export/efficientdet.tflite') # 讀取 .tflite 模型
interpreter.allocate_tensors()                                   # 初始化模型
 
input_details = interpreter.get_input_details()                  # 獲取輸入點訊息
output_details = interpreter.get_output_details()                # 獲取輸出點訊息


w,h,c=input_details[0]['shape'][1:4]           # 獲取輸入圖片大小
img_file='/content/drive/MyDrive/Colab Notebooks/efficientdet/20201110_222144_002.jpg' # 一個圖片路徑
imgc = 0
Pred = 0


img=cv.imread(img_file)                        # 讀取圖片
img=img.astype(np.uint8)                       # 轉整數
img=cv.resize(img,(w,h))                       # 調整圖片大小
img=cv.cvtColor(img,cv.COLOR_BGR2RGB)          # BGR to RGB
img=img.reshape((w,h,c))                       # 調整陣列大小
img=np.expand_dims(img,0)                      # 轉 4D 陣列

interpreter.set_tensor(input_details[0]['index'], img)                 # 輸入資料到模型
interpreter.invoke()                                                   # 模型計算輸出

detection_scores = interpreter.get_tensor(output_details[0]['index']) # 獲取輸出各框的得分
detection_boxes = interpreter.get_tensor(output_details[1]['index']) # 獲取輸出框座鰾 (百分比)
num_boxes = interpreter.get_tensor(output_details[2]['index'])  # 獲取輸出總框數量
detection_classes = interpreter.get_tensor(output_details[3]['index']) # 輸出框類別 

#print(detection_scores)

num_boxes=int(num_boxes[0])                                                   # 取得整數值
detection_boxes=detection_boxes.reshape(num_boxes,4)                          # 調整陣列為10x4大小
detection_classes=detection_classes.astype(dtype=np.int).reshape(num_boxes)+1 # 類別為整數值 (+1跳過背景類)
detection_scores=detection_scores.reshape(num_boxes)                          # 調整陣列為10大小
#print(detection_boxes)              #框座標
#print(detection_classes)            #各個框類別
#print(detection_scores)             #框分數
#print(num_boxes)                    #總共的框
# 讀取圖片並畫框

outimg=cv.imread(img_file)
for i in range(0,num_boxes):
    # 把百分比座標轉為實際圖片座標
    detection_boxes[i,0]=int(detection_boxes[i,0]*outimg.shape[0])
    detection_boxes[i,1]=int(detection_boxes[i,1]*outimg.shape[1])
    detection_boxes[i,2]=int(detection_boxes[i,2]*outimg.shape[0])
    detection_boxes[i,3]=int(detection_boxes[i,3]*outimg.shape[1])
    # 得分大於 0.5 (可自行調整) 才畫出框及文字敘述

    if detection_scores[i]>0.5:
        outimg=cv.rectangle(outimg, (int(detection_boxes[i,1]), int(detection_boxes[i,0])), (int(detection_boxes[i,3]), int(detection_boxes[i,2])), (0,0,255),2)
        if detection_classes[i] ==1:
            ID = 'Mouse'
        elif detection_classes[i] ==2:
            ID = 'Keyboard'
        elif detection_classes[i] ==3:
            ID = 'Screen'
        outimg=cv.putText(outimg,'ID: %s'%(ID), (int(detection_boxes[i,1]),int(detection_boxes[i,0]-3)),cv.FONT_HERSHEY_COMPLEX  ,1.5,(0,0,0),2)

cv.imwrite('out.jpg',outimg)
'''      
cv.imshow('out', outimg)
key = cv.waitKey(0)
if key>0:
    cv.destroyAllWindows()
'''